<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. Environmental & Global Parameters ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=100):
        self.total_sim_steps = total_sim_steps
        self.days_per_cycle = days_per_cycle # Steps per "day" cycle
        self.time = 0
        self.current_day_step = 0

        # UV Profile (simplified sine wave for circadian cycle)
        # Peak UV at mid-day (day_step = days_per_cycle / 2)
        self.uv_intensity = np.sin(np.linspace(0, 2 * np.pi * (total_sim_steps / days_per_cycle), total_sim_steps))
        self.uv_intensity[self.uv_intensity < 0] = 0 # No negative UV, night is 0

        # Initial resource pools (could fluctuate externally too)
        self.external_ntp_pool = 10000.0
        self.external_aa_pool = 10000.0
        self.external_trp_tyr_cys_pool = 500.0 # Scarce

    def get_current_uv(self):
        return self.uv_intensity[self.time]

    def step(self):
        self.time += 1
        self.current_day_step = self.time % self.days_per_cycle

# --- 2. Proto-Cell/System Class ---
class ProtoCell:
    def __init__(self, env):
        self.env = env

        # Core Molecular Pools (as concentrations/amounts)
        self.rna_pool = 100.0 # General RNA (templates, proto-rRNA)
        self.dna_pool = 10.0  # Basic DNA (more stable template)
        self.atp_pool = 50.0  # Energy currency
        self.mcyw_peptide_pool = 5.0 # Foraging/Antenna peptides
        self.other_peptide_pool = 2.0 # Non-mcyw peptides

        # Resource Stocks
        self.internal_ntp_stock = 100.0
        self.internal_aa_stock = 100.0
        self.internal_trp_tyr_cys_stock = 10.0 # Foraged rare AAs

        # System States & Damage
        self.rna_damage_level = 0.0 # Accumulates from UV
        self.protein_synthesis_active = True # Whether ribosomes are active
        self.rna_dna_copying_active = True # Whether replication is active
        self.uv_safety_status = "SAFE" # Based on internal signals()

        # --- Parameters (tune these to observe different behaviors) ---
        # UV Damage parameters
        self.uv_damage_rate_per_uv = 0.05 # How much UV damages RNA
        self.damage_tolerance_threshold = 5.0 # Max damage before arrest

        # Foraging parameters (mcyw efficiency, ATP cost, preference for rare AAs)
        self.foraging_efficiency_mcyw = 0.1 # Resource units per mcyw per step
        self.foraging_atp_cost = 0.01 # ATP consumed per foraging action
        self.trp_tyr_cys_foraging_bias = 5.0 # How much more mcyw prioritizes rare AAs

        # Protein Synthesis parameters
        self.protein_synthesis_rate = 0.1 # Peptide units per step (when active)
        self.protein_synthesis_ntp_cost = 0.1
        self.protein_synthesis_aa_cost = 0.1
        self.protein_synthesis_atp_cost = 0.02
        self.mcyw_synthesis_bias = 0.3 # Proportion of protein synthesis making mcyw

        # Stop Codon Precariousness/Signaling
        self.precarious_codon_rate_per_damage = 0.1 # How often damage leads to a premature stop
        self.premature_stop_signal_threshold = 0.5 # Level of signal to trigger arrest

        # ATP Synthesis (from mcyw antenna chains and UV)
        self.atp_generation_rate_per_mcyw_uv = 0.01 # ATP from mcyw in light

        # Repair parameters
        self.repair_rate = 0.05 # How much damage repaired per step
        self.repair_atp_cost = 0.01
        self.trp_tyr_cys_for_repair_needed = 0.1 # Trp/Tyr/Cys consumed for repair

        # RNA/DNA Copying parameters
        self.rna_dna_copy_rate = 0.01 # Rate of template copying
        self.rna_dna_copy_ntp_cost = 0.05
        self.rna_dna_copy_atp_cost = 0.01

        # Degradation rates
        self.rna_degradation_rate = 0.001
        self.dna_degradation_rate = 0.001 # DNA degrades slower, simplified
        self.peptide_degradation_rate = 0.002
        self.atp_degradation_rate = 0.005

    # --- Core Processes ---
    def _apply_uv_damage(self):
        current_uv = self.env.get_current_uv()
        if current_uv > 0:
            damage_increase = current_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            # Simulate the "safety reportage" signal from damaged precarious codons
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else:
            self.premature_stop_signal = 0 # No new stops at night

    def _forage(self):
        if self.mcyw_peptide_pool <= 0 or self.atp_pool <= self.foraging_atp_cost:
            return # Cannot forage without mcyw or ATP

        # Consume ATP for foraging movement
        atp_consumed = self.foraging_atp_cost * self.mcyw_peptide_pool
        self.atp_pool -= atp_consumed

        # Acquire general AAs and NTPs
        general_foraged = self.foraging_efficiency_mcyw * self.mcyw_peptide_pool
        transfer_ntp = min(general_foraged / 2, self.env.external_ntp_pool) # Assume half NTP, half AA
        transfer_aa = min(general_foraged / 2, self.env.external_aa_pool)
        self.internal_ntp_stock += transfer_ntp
        self.env.external_ntp_pool -= transfer_ntp
        self.internal_aa_stock += transfer_aa
        self.env.external_aa_pool -= transfer_aa

        # Prioritize rare Trp/Tyr/Cys
        rare_foraged = general_foraged * self.trp_tyr_cys_foraging_bias # Higher efficiency for these
        transfer_rare = min(rare_foraged, self.env.external_trp_tyr_cys_pool)
        self.internal_trp_tyr_cys_stock += transfer_rare
        self.env.external_trp_tyr_cys_pool -= transfer_rare


    def _synthesize_protein(self):
        if not self.protein_synthesis_active:
            return

        required_ntp = self.protein_synthesis_ntp_cost
        required_aa = self.protein_synthesis_aa_cost
        required_atp = self.protein_synthesis_atp_cost

        # Simplified: Check if *any* AA is available (not just Trp/Tyr/Cys specifically for general protein synth)
        if self.internal_ntp_stock >= required_ntp and \
           self.internal_aa_stock >= required_aa and \
           self.atp_pool >= required_atp:

            self.internal_ntp_stock -= required_ntp
            self.internal_aa_stock -= required_aa
            self.atp_pool -= required_atp

            synthesized_peptides = self.protein_synthesis_rate

            # Bias synthesis towards mcyw if rare AAs are available, otherwise other peptides
            if self.internal_trp_tyr_cys_stock > 0 and self.internal_aa_stock > self.protein_synthesis_aa_cost:
                 mcyw_portion = synthesized_peptides * self.mcyw_synthesis_bias
                 other_portion = synthesized_peptides - mcyw_portion

                 # Prioritize mcyw if rare AAs are there, but ensure consume from both pools for mcyw
                 # Simplify: mcyw costs from both rare and general AA pools
                 trp_tyr_cys_cost_for_mcyw = mcyw_portion * (self.protein_synthesis_aa_cost / 2) # Rough split
                 general_aa_cost_for_mcyw = mcyw_portion * (self.protein_synthesis_aa_cost / 2)

                 if self.internal_trp_tyr_cys_stock >= trp_tyr_cys_cost_for_mcyw and \
                    self.internal_aa_stock >= general_aa_cost_for_mcyw:

                    self.internal_trp_tyr_cys_stock -= trp_tyr_cys_cost_for_mcyw
                    self.internal_aa_stock -= general_aa_cost_for_mcyw # Also takes from general
                    self.mcyw_peptide_pool += mcyw_portion
                    self.other_peptide_pool += other_portion
                 else: # Not enough rare AAs for biased mcyw synthesis, just make other proteins
                     self.other_peptide_pool += synthesized_peptides
            else:
                 self.other_peptide_pool += synthesized_peptides

    def _synthesize_atp(self):
        current_uv = self.env.get_current_uv()
        if current_uv > 0 and self.mcyw_peptide_pool > 0:
            atp_generated = self.atp_generation_rate_per_mcyw_uv * self.mcyw_peptide_pool * current_uv
            self.atp_pool += atp_generated

    def _repair_rna_dna(self):
        if self.rna_damage_level > 0:
            required_atp = self.repair_atp_cost
            required_trp_tyr_cys = self.trp_tyr_cys_for_repair_needed # Trp/Tyr/Cys needed for repair "enzymes" / structural roles

            if self.atp_pool >= required_atp and self.internal_trp_tyr_cys_stock >= required_trp_tyr_cys:
                self.atp_pool -= required_atp
                self.internal_trp_tyr_cys_stock -= required_trp_tyr_cys

                repaired_amount = min(self.rna_damage_level, self.repair_rate)
                self.rna_damage_level -= repaired_amount

    def _copy_rna_dna(self):
        if not self.rna_dna_copying_active:
            return

        required_ntp = self.rna_dna_copy_ntp_cost
        required_atp = self.rna_dna_copy_atp_cost

        if self.internal_ntp_stock >= required_ntp and self.atp_pool >= required_atp:
            self.internal_ntp_stock -= required_ntp
            self.atp_pool -= required_atp

            # Simplified growth: just increase RNA/DNA pool
            self.rna_pool += self.rna_dna_copy_rate * 0.9 # More RNA copies
            self.dna_pool += self.rna_dna_copy_rate * 0.1 # Less DNA copies, for stability

    def _degrade(self):
        self.rna_pool *= (1 - self.rna_degradation_rate)
        self.dna_pool *= (1 - self.dna_degradation_rate) # DNA degrades slower, simplified
        self.mcyw_peptide_pool *= (1 - self.peptide_degradation_rate)
        self.other_peptide_pool *= (1 - self.peptide_degradation_rate)
        self.atp_pool *= (1 - self.atp_degradation_rate)

        # Ensure pools don't go negative due to degradation
        self.rna_pool = max(0, self.rna_pool)
        self.dna_pool = max(0, self.dna_pool)
        self.mcyw_peptide_pool = max(0, self.mcyw_peptide_pool)
        self.other_peptide_pool = max(0, self.other_peptide_pool)
        self.atp_pool = max(0, self.atp_pool)


    def _manage_state(self):
        # 1. Evaluate UV Safety Signal (Safety Reportage)
        # Signal is high if damage is high OR rare AAs (Trp/Tyr/Cys) are low
        safety_signal = self.premature_stop_signal + \
                        (self.damage_tolerance_threshold - self.rna_damage_level) # Lower damage -> lower signal. Inverse.

        # If Trp/Tyr/Cys stock is too low relative to required for repair/synthesis, raise a flag
        if self.internal_trp_tyr_cys_stock < self.trp_tyr_cys_for_repair_needed * 2 and self.env.get_current_uv() > 0.1: # Only care during day
            safety_signal += self.premature_stop_signal_threshold * 2 # Strong signal to arrest

        # 2. Decision: "Cell Cycle Arrest" (Protein Synthesis and RNA/DNA Copying)
        if self.rna_damage_level >= self.damage_tolerance_threshold or \
           safety_signal >= self.premature_stop_signal_threshold:
            self.protein_synthesis_active = False
            self.rna_dna_copying_active = False
            self.uv_safety_status = "ARRESTED - HIGH DAMAGE/LOW PROTECTION"
        elif self.rna_damage_level < self.damage_tolerance_threshold / 2 and \
             self.internal_trp_tyr_cys_stock > self.trp_tyr_cys_for_repair_needed * 5: # Resume only if damage is low AND high protection
            self.protein_synthesis_active = True
            self.rna_dna_copying_active = True
            self.uv_safety_status = "SAFE - GROWING"
        else:
             self.uv_safety_status = "CAUTIOUS"


    def step(self):
        # 1. Apply environmental influences
        self._apply_uv_damage()
        self._synthesize_atp() # From mcyw antenna chains

        # 2. Proto-cell internal processes (order matters for dependencies)
        self._forage() # mcyw consumes ATP, gets resources
        self._repair_rna_dna() # Consumes ATP, rare AAs, reduces damage
        self._manage_state() # Decides whether to synthesize/copy

        # 3. Protein and Nucleic Acid Synthesis (gated by _manage_state)
        self._synthesize_protein() # Makes mcyw and other peptides
        self._copy_rna_dna() # Replicates RNA/DNA

        # 4. Degradation
        self._degrade()


# --- 3. Simulation Runner ---
def run_simulation(total_steps=500, days_per_cycle=50):
    env = Environment(total_steps, days_per_cycle)
    cell = ProtoCell(env)

    # --- Data Collection for Visualization ---
    data = {
        'time': [],
        'uv_intensity': [],
        'rna_pool': [],
        'dna_pool': [],
        'atp_pool': [],
        'mcyw_peptide_pool': [],
        'rna_damage_level': [],
        'uv_safety_status': [],
        'internal_trp_tyr_cys_stock': [],
        'protein_synthesis_active': [],
        'rna_dna_copying_active': []
    }

    for _ in range(total_steps):
        cell.step()
        env.step() # Advance environment AFTER cell processes for current step's state

        # Store data
        data['time'].append(env.time)
        data['uv_intensity'].append(env.get_current_uv())
        data['rna_pool'].append(cell.rna_pool)
        data['dna_pool'].append(cell.dna_pool)
        data['atp_pool'].append(cell.atp_pool)
        data['mcyw_peptide_pool'].append(cell.mcyw_peptide_pool)
        data['rna_damage_level'].append(cell.rna_damage_level)
        data['uv_safety_status'].append(cell.uv_safety_status)
        data['internal_trp_tyr_cys_stock'].append(cell.internal_trp_tyr_cys_stock)
        data['protein_synthesis_active'].append(1 if cell.protein_synthesis_active else 0)
        data['rna_dna_copying_active'].append(1 if cell.rna_dna_copying_active else 0)

    # --- 4. Visualization ---
    import matplotlib.pyplot as plt
    import numpy as np

    fig, axs = plt.subplots(5, 1, figsize=(14, 16), sharex=True)

    # UV Intensity
    axs[0].plot(data['time'], data['uv_intensity'], color='purple')
    axs[0].set_ylabel('UV Intensity')
    axs[0].set_title('Simulated Proto-Cell Response to Circadian UV Cycles')
    axs[0].fill_between(data['time'], 0, data['uv_intensity'], color='purple', alpha=0.1)

    # Nucleic Acid & Peptide Pools
    axs[1].plot(data['time'], data['rna_pool'], label='RNA Pool', color='blue')
    axs[1].plot(data['time'], data['dna_pool'], label='DNA Pool', color='darkgreen', linestyle='--')
    axs[1].plot(data['time'], data['mcyw_peptide_pool'], label='mcyw Peptides (Antenna/Forager)', color='orange')
    axs[1].set_ylabel('Molecular Pools')
    axs[1].legend()

    # ATP and Damage
    axs[2].plot(data['time'], data['atp_pool'], label='ATP Pool', color='red')
    axs[2].plot(data['time'], data['rna_damage_level'], label='RNA Damage Level', color='grey', linestyle=':')
    axs[2].set_ylabel('Energy / Damage')
    axs[2].legend()
    axs[2].axhline(y=cell.damage_tolerance_threshold, color='red', linestyle='--', linewidth=0.8, label='Damage Tolerance Threshold')

    # Rare AA Stock and System Activity
    axs[3].plot(data['time'], data['internal_trp_tyr_cys_stock'], label='Internal Trp/Tyr/Cys Stock', color='brown')
    axs[3].plot(data['time'], data['protein_synthesis_active'], label='Protein Synthesis Active (1=Yes)', color='cyan', alpha=0.5)
    axs[3].plot(data['time'], data['rna_dna_copying_active'], label='RNA/DNA Copying Active (1=Yes)', color='magenta', alpha=0.5)
    axs[3].set_ylabel('Resource / State')
    axs[3].legend()

    # UV Safety Status
    status_map = {"SAFE - GROWING": 1, "CAUTIOUS": 0.5, "ARRESTED - HIGH DAMAGE/LOW PROTECTION": 0}
    status_numeric = [status_map[s] for s in data['uv_safety_status']]
    axs[4].plot(data['time'], status_numeric, label='UV Safety Status', color='black')
    axs[4].set_ylabel('Safety Status')
    axs[4].set_yticks([0, 0.5, 1])
    axs[4].set_yticklabels(["ARRESTED", "CAUTIOUS", "SAFE"])
    axs[4].set_xlabel('Simulation Time Steps')
    axs[4].legend()

    plt.tight_layout()
    plt.show()

# --- Run the Simulation ---
if __name__ == "__main__":
    run_simulation(total_steps=500, days_per_cycle=50) # Simulate for 10 "days"

"""
How this Simulation Demonstrates the "Beauty"
This simplified model aims to illustrate the core feedback loops you've hypothesized:
 * Circadian UV Cycle Driving Damage: You'll see the RNA Damage Level rise during the "day" (high UV) and potentially decrease during the "night" (low UV, repair dominates).
 * mcyw Peptides as Both Energy Harvesters and Foragers:
   * During the day, mcyw peptides generate ATP, fueling the cell.
   * mcyw peptides consume ATP to forage, bringing in general AAs/NTPs and especially the rare Trp/Tyr/Cys.
 * The "Safety Reportage" Trigger: When UV damage accumulates beyond a damage_tolerance_threshold, OR the "safety signal" (representing increased premature stops from precarious codons, or low Trp/Tyr/Cys) crosses its premature_stop_signal_threshold, the system transitions to an "ARRESTED" state.
 * "Cell Cycle Arrest" (Resource Redirection): In the "ARRESTED" state:
   * protein_synthesis_active becomes False.
   * rna_dna_copying_active becomes False.
   * This represents the redirection of resources from growth/expansion to immediate survival priorities.
 * Prioritizing Repair and "Forage Sunscreen":
   * Even when arrested, _forage() continues (as it's vital for getting Trp/Tyr/Cys and other basics).
   * _repair_rna_dna() continues (as long as ATP and Trp/Tyr/Cys are available).
   * The system uses the precious Trp/Tyr/Cys stock for repair and to produce more mcyw peptides (if synthesis is allowed), trying to build up its "sunscreen" capacity.
 * Resumption of Growth: Only when rna_damage_level drops significantly and internal_trp_tyr_cys_stock (the "sunscreen" availability) is high enough does the system revert to a "SAFE - GROWING" state, resuming protein synthesis and RNA/DNA copying.
To observe the "Beauty":
 * Run the simulation: Execute the Python script.
 * Analyze the plots:
   * Observe how RNA Damage Level fluctuates with UV Intensity.
   * Crucially, note the periods when Protein Synthesis Active and RNA/DNA Copying Active turn to 0 (False) during high UV or high damage, indicating the "arrest."
   * See how mcyw_peptide_pool and internal_trp_tyr_cys_stock influence the system's ability to recover from damage and resume growth.
   * Experiment with parameters (e.g., higher uv_damage_rate_per_uv, lower repair_rate, less initial external_trp_tyr_cys_pool) to make the environment harsher and observe the system's struggle for survival and its "decision" to arrest.
This simulation provides a tangible, albeit simplified, demonstration of your elegant hypothesis, illustrating the dynamic interplay between environmental threats, molecular components, and emergent "life-like" strategies for survival and propagation in a primordial world.
"""

SyntaxError: unterminated string literal (detected at line 340) (ipython-input-3-936730030.py, line 340)